Transport phenomena is the study of the transferring of momentum, energy and mass. Oftentimes these processes are coupled as in boiling water, where bubbles of air trapped in the water nucleate due to the elevated temperature reducing the solubility of gases, followed by the density differences between the bubble and water causing the bubble to rise. These processes are further mediated through convection currents caused by temperature gradients in the boiling water. 

In this example, we will look at a far simple example of energy transport, characterized as the change in temperature of a heat source placed in an infinitely large cooler surrounding region. We will assume periodic boundary conditions to overcome the limitation of finite sizes in compute and assume that the temperature in the source reduces at the same time.  (INSERT IMAGE HERE)

# Theory

For our problem, we will be basing our grid on rectilinear or cartesian coordinates. First, we will review the relevant transport equations. A more detailed explanation of each expression and derivation can be found in BSLK. We begin by writing a general energy transport equation through a control volume.

$$ change = accumulation + removal + generation + destruction $$

In our closed system, there is no generation, destruction and accumulation of energy, meaning that the change in energy over time can be written as

$ change = removal$

If we approximate that the energy lost from the addition of heat is lost through conduction, the removal of energy can be approximated using fourier's law, defined as 

$$ q = -k \nabla T $$

where $k (\frac{W}{mK})$ is the thermal conductivity of the matrix, $\nabla T (\frac{K}{m})$ is the temperature gradient between the heated area and its surroundings and $q (\frac{W}{m^2 K})$ is the heat flux through a surface. This is related to the energy change at each grid point over time as, 

$\frac{d E}{dt} = \nabla q$

Where $E (\frac{J}{m^3})$ is the energy density of a specific grid point and $t (s)$ is the time. This leads to the expression, 

$$ \frac{d E}{d t} = -k \nabla^2 T $$

The energy density of the heat source can be expressed from the heat capacity of the heat source, defined as $E = \rho c_p (T - T_{ref})$. If we make the further approximation that density $\rho (\frac{kg}{m^3})$ and specific heat capacity $\frac{J}{K kg}$ stay constant with temperature, we can rearrange the above expression to obtain the following PDE written as a function of temperature exclusively,

$$ \frac{dT}{dt} = -\frac{k}{\rho c_p} \nabla^2 T$$

Oftentimes, the factor $\frac{k}{\rho c_p}$ is condensed into the term $\alpha (\frac{m^2}{s})$ defining the thermal diffusivity of the system and will be used as an input parameter.

Now that we have laid out the basic theory being implemented, we shall turn to how we can convert math to something a computer can solve. There are derivatives of time and space that we will need to solve for. I will stick to the simplest ones here as the focus is on creating an illustration how the code looks different between implementations in different languages. However, the literature of how both are done is vast with (SOURCES PROVIDED).

## Finite difference

The simplest way to discretize a differential equation in space is using a finite difference technique. The core idea of this scheme is we convert the continous differential equations above into discrete sums and differences which we can solve. Commonly, this is done using a central difference scheme shown below for the first $f^1_x$ and second derivative $f^2_x$ for a function $f(x,y,t)$

\begin{align}
f^1_x &= \frac{f(x + h, y, t) - f(x - h, y, t)}{2h} \\
f^2_x &= \frac{f(x + h, y, t) - 2f(x,y,t) + f(x - h, y, t)}{h^2} 
\end{align}

Where $h$ is the spacing between grids. Central difference offers produced second order error compared to forward or backward difference which produce first order error.

## Forward Euler

Next, I look at how we can integrate over time. Multiple time integration schemes exist which are split into explicit and implicit schemes. Explicit schemes include techniques such as Forward Euler and the Runge-Kutta time integrators while implicit schemes include the Backwards Euler and Crank-Nicholson schemes. For now, we will stick to the Forward Euler technique. 

Forward Euler solves an initial value problem by iterating a value of a variable through time using the initial value and the change in the variable through time. Notationally, this looks like

\begin{align}
g(x, y, t_0) &= b \\
\frac{dg}{dt} &= f(x,y,t) \\
g(x, y , t + \Delta t) &= g(x, y, t) + \Delta t \frac{dg}{dt}
\end{align}

where $g(x, y, t)$ is a value of a function at time $t$ and location $(x,y)$, $\frac{dg}{dt}$ is the derivative of function $g$ with respect to time $t$ and is expressed as any function $f(x,y,t)$. The value of $g(x, y, t + \Delta t)$ is a sum of the value of $g(x, y, t)$ and the timestep over which the derivative $\frac{dg}{dt}$ is calculated.

The Forward Euler technique can be numerically unstable. Therefore care must be taken when selecting parameters for time and space integration and must fall within the CFL condition, defined as $C = \frac{u \Delta t}{\Delta x} < 1$.

## Python implementation

Using the established rules above, we move to generating a prototype in python. We assume the heat source places is a circle at the middle of the system with a radius that can be specified with the system containing periodic boundary conditions. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def animate_colormap(data, axs_labels=None, times=None, c_label=None, interval=50, sz=5, cm='bwr'):
    """
    Create an animation of a 2D colormap over time.

    This function takes a 3D numpy array representing time-dependent 2D data and 
    generates an animated colormap. The animation can be rendered in a Jupyter 
    notebook using the :class:`IPython.display.HTML` object.

    :param data: 
        A 3D array with shape (t, L, M), where t is the number of timesteps, 
        and L and M represent the dimensions of each data slice (e.g., rows and columns).
    :type data: numpy.ndarray
    :param axs_labels: 
        A list of length 2 containing strings for the x-axis and y-axis labels, 
        in the 0th and 1st positions respectively. Default is None.
    :type axs_labels: list of str, optional
    :param times: 
        A 1D array of length t, where each value represents the time corresponding 
        to each timestep. Default is None.
    :type times: numpy.ndarray, optional
    :param c_label: 
        A label for the colormap (color bar). Default is None.
    :type c_label: str, optional
    :param interval: 
        The delay in milliseconds between frames in the animation. Default is 50.
    :type interval: int, optional
    :param sz: 
        The size of the plot figure. Default is 5.
    :type sz: int, optional
    :param cm: 
        The colormap to use for the animation. Default is 'bwr'.
    :type cm: str, optional

    :return: 
        An animation object that can be rendered in a Jupyter notebook using 
        :class:`IPython.display.HTML`.
    :rtype: matplotlib.animation.FuncAnimation

    :examples:
        >>> import numpy as np
        >>> from IPython.display import HTML
        >>> data = np.random.random((50, 100, 100))  # Example 3D array (50 timesteps, 100x100 grid)
        >>> ani = animate_colormap(data, axs_labels=["X-axis", "Y-axis"], times=np.arange(50))
        >>> HTML(ani.to_jshtml())
    """
    def init():
        img.set_data(data[0])
        vmin = np.amin(data[0])
        vmax = np.amax(data[0])
        img.set_clim(vmin, vmax)
        if times is not None:
            ax.set(title=f"Time = {times[0]}")
        return (img,)

    def update(i):
        img.set_data(data[i])
        vmin = np.amin(data[i])
        vmax = np.amax(data[i])
        img.set_clim(vmin, vmax)
        if times is not None:
            ax.set(title=f"Time = {times[i]}")
        return (img,)

    fig, ax = plt.subplots(1, 1, figsize=(sz, sz))
    img = ax.imshow(data[0], cmap=cm, vmin=np.amin(data[0]), vmax=np.amax(data[0]))
    if axs_labels is not None:
        ax.set_xlabel(axs_labels[0])
        ax.set_ylabel(axs_labels[1])
    fig.colorbar(img, ax=ax, orientation="horizontal", label=c_label, pad=0.2)
    ani = animation.FuncAnimation(fig, update, frames=len(data), init_func=init, interval=interval, blit=True)
    plt.close()
    return ani

In [ ]:
def make_system(nx, ny, R, T_hot = 373, T_cold = 273):
    X, Y = np.meshgrid(*[np.arange(0, nx), np.arange(0, ny)])
    out = np.where((X - nx/2)**2 + (Y - ny/2)**2 <= R**2, T_hot, T_cold)
    return out

def FD_timestep(grid, dx, dy, dt, alpha, halo):
    new_grid = grid.copy()
    nx, ny = grid.shape # current shape includes the halo points on the edge of the grid

    slc = np.s_[halo:nx-halo, halo:ny-halo] # array slicing for non edge slices
    
    # central difference scheme for laplacian in x direction for spatial integration
    central_difference_x = (grid[halo+1:nx-halo+1, halo:ny-halo] - 2*grid[halo:nx-halo, halo:ny-halo] + grid[halo-1:nx-halo-1, halo:ny-halo])/dx**2
    # central difference scheme for laplacian in y direction for spatial integration
    central_difference_y = (grid[halo:nx-halo, halo+1:ny-halo+1] - 2*grid[halo:nx-halo, halo:nx-halo] + grid[halo:nx-halo, halo-1:nx-halo-1])/dy**2
    
    # performing forward euler for time integration for non halo regions
    new_grid[slc] = grid[slc] + alpha*dt*(central_difference_x + central_difference_y)

    return new_grid

def periodic_boundary(grid, halo):
    nx, ny = grid.shape # current shape includes the halo points on the edge of the grid
    new_grid = grid.copy()

    # periodic boundary for y direction copies the values in the other side of the 'real' region to the halo
    new_grid[halo:halo+halo,:] = grid[nx-halo*halo:nx-halo*halo+halo, :]
    new_grid[nx-halo*halo:nx-halo*halo+halo,:] = grid[halo:halo+halo, :]

    # periodic boundary for x direction copies the values in the other side of the 'real' region to the halo
    new_grid[:,halo:halo+halo] = grid[:,ny-halo*halo:ny-halo*halo+halo]
    new_grid[:,ny-halo*halo:ny-halo*halo+halo] = grid[:,halo:halo+halo]

    return new_grid

In [ ]:
## CHANGE THESE ##
nx = 200 
ny = 200
# keep square for now (nx == ny)
halo = 2
R = 30

timesteps = 500
dt = 0.001
dump_freq = 10

alpha = 0.001
T_hot = 373
T_cold = 273
## CHANGE THESE ##

dx = 1/nx
dy = 1/ny
C = alpha/(min(dx, dy)**2/dt)
convergence_crit = 1e-7
if C > 1:
    raise RuntimeWarning("CFL condition not fulfilled")

temperature_fields_time = np.empty((timesteps//dump_freq+1, nx, ny))
times = np.empty(timesteps//dump_freq+1)

grid_old = make_system(nx+2*halo, ny+2*halo, R)
grid_new = np.empty((nx+2*halo, ny+2*halo))

for time in range(0, timesteps+1):
    if time%dump_freq == 0:
        temperature_fields_time[time//dump_freq] = grid_old[halo:nx+halo, halo:ny+halo]
        times[time//dump_freq] = time
    
    grid_new = FD_timestep(grid_old, 1, 1, dt, alpha, halo)
    grid_new = periodic_boundary(grid_new, halo)

    diff = np.mean(grid_new - grid_old)
    if np.abs(diff) < convergence_crit:
        temperature_fields_time[time//dump_freq + 1] = grid_new[halo:nx+halo, halo:ny+halo]
        times[time//dump_freq+1] = time

        temperature_fields_time = temperature_fields_time[:time//dump_freq + 1, :, :]
        times = times[:time//dump_freq+1]
        print("Converged")
        break

    grid_old = grid_new.copy()

In [ ]:
ani = animate_colormap(temperature_fields_time, times = times)
HTML(ani.to_jshtml())

In [ ]:
plt.imshow(temperature_fields_time[-1])
plt.colorbar()

# Single core implementation in C++

In [ ]:
def read_data(file_path):
    data = []
    with open(file_path, "r") as f:
        list_of_text = f.readlines()
        for curr_line in list_of_text:
            curr_data = [float(i) for i in curr_line.split(",")[:-1]]
            data.append(curr_data)

    data = np.array(data)

    return data

# Multicore (CPU) implementation in C++

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
path = "src"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))

In [ ]:
data = read_data(data_paths[-1])
plt.imshow(data)
plt.colorbar()